In [1]:
import json
import numpy as np
import pandas as pd
import re

In [2]:
data = []
with open("frontend/test.json", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data.append(json.loads(line))
        
data = pd.DataFrame(data)
data.head()

,id,last_analyzed,document_id,size,content_type,doc_title,agenda_item_id,agenda_item_title,agenda_item_text,meeting_id,...,creation_date,last_modified,last_saved,doc_type,consultation_id,consultation_name,consultation_type,consultation_text,consultation_topic,consultation_organization
0,1,2023-05-06T14:02:29Z,6164,169369,application/pdf,Stvv01 - 14.04.2011,[],[],[],[1413],...,2017-11-30T12:08:04Z,2017-11-30T12:08:04Z,2017-11-30T12:08:04Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2023-05-06T14:02:29Z,553,99102,application/pdf,1 Pro-JSKS,[],[],[],[69],...,2016-05-26T05:49:28Z,2016-05-26T05:49:28Z,2016-05-26T05:49:28Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2023-05-06T14:02:29Z,722,183796,application/pdf,Kostenplan 2013-12 Mai 2016,[],[],[],[76],...,2016-05-10T07:46:01Z,2016-05-10T09:32:42Z,2016-05-10T09:32:42Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2023-05-06T14:02:29Z,3961,331258,application/pdf,Niederschrift SB,[],[],[],[1220],...,2017-03-20T14:30:09Z,2017-03-20T14:30:09Z,2017-03-20T14:30:09Z,Niederschrift,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2023-05-06T14:02:29Z,723,267877,application/pdf,2 Pro-Schw,[],[],[],[76],...,2016-08-12T09:23:49Z,2016-08-12T09:23:49Z,2016-08-12T09:23:49Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.doc_type.unique()

array([nan, 'Niederschrift', 'Beschlussvorlage', 'Antragsvorlage',
       'Informationsvorlage'], dtype=object)

In [4]:
def iterate_content(doc_type=None, document_ids=None):
    if doc_type:
        filtered = data[data.doc_type == doc_type]
    else:
        filtered = data
    
    if document_ids:
        filtered = filtered[np.isin(filtered.document_id, document_ids)]
    
    for (document_id, content) in zip(filtered["document_id"], filtered["content"]):
        yield (document_id, content)
        

def remove_by_regexes(content, regexes):
    """ Removes the header of documents based on given regex patterns.
    The content lines are scanned for the first occurance of all regexes and the returned output
    contains only the content after the last regex match.
    Therefore, if a regex is found twice, only the first match is considered.
    If two regexes match, the content is cut for the last match of the two.
    """
    matches = []  # tuples (line, regex_end) for all first matches
    for regex in regexes:
        for idx, line in enumerate(content):
            res = re.search(regex, line)
            if res:
                matches.append((idx, res.end()))
                break

    last_match_line = np.max([match[0] for match in matches])
    last_match_inline = 0
    for line, inline in matches:
        if line != last_match_line:
            continue
        last_match_inline = np.max([last_match_inline, inline])
        
    content = content[last_match_line:]
    content[0] = content[0][last_match_inline:]
    if content[0].strip == "":
        content = content[1:]
    return content

# test remove_by_regexes:
result = remove_by_regexes([
    "first line",
    "second line with match in middle",
    "third line with match in middle",
    "fourth line"
], [r"with match", "third line"])

assert result == [" with match in middle", "fourth line"]

# Antragsvorlage

In [5]:
# create a test dataset for antrag header removal test
antraege = data[data.doc_type == "Antragsvorlage"]
print("dataset size", len(antraege))
data_set_identifiers = [
    lambda x: "die Stadtverordnetenversammlung möge beschließen:" in x and x.endswith(":"),
    lambda x: "die Stadtverordnetenversammlung möge beschließen" in x and "beschließen:" not in x,
    lambda x: "die Stadtverordnetenversammlung möge beschließen" in x and len(x) > 100,
    lambda x: "Wir beantragen. die Stadtverordnetenversammlung möge beschließen" in x,
    lambda x: "bitte nehmen Sie den folgenden Antrag zur Beratungen auf die Tagesordnung" in x,
]
selected_ids = []
ids = []
for identifier in data_set_identifiers:
    for id, content in zip(antraege.document_id.values, antraege.content.values):
        for line in content:
             if identifier(line):
                ids.append(id)
                break
    selected_ids += np.random.choice(ids, size=1).tolist()


print(f"test set size {len(selected_ids)}")

dataset size 228
test set size 5


In [6]:
selector = np.isin(antraege.document_id, selected_ids)
for doc_id, content in iterate_content(doc_type="Antragsvorlage", document_ids=selected_ids):
    print(doc_id)
    result = remove_by_regexes(content, [
        r"die Stadtverordnetenversammlung möge beschließen[\s\:\.,]*",
        r"Sehr geehrte[r]? (Herr|Frau) Stadtverordnetenvorsteher(in)?[\s\.,]*",
    ])
    print("".join(result)[:100], "...")
    print("")
    print("\n".join(content))

    print("=-----------------------------------------==")

23449
Die Straßenbeitragssatzung der Stadt Griesheim vom 14.12.2012 wird aufgehoben. Eine neue Satzung wir ...

SPD-Stadtverordnetenfraktion Griesheim
SPD-Stadtverordnetenfraktion Ingrid Zimmermann, Goethestr. 30, 64347 Griesheim
Frau Stadtverordnetenvorsteherin Martina Bott Wilhelm-Leuschner-Str. 75 64347 Griesheim
:.-.
•
11. MAI 2018
dtverordnet.envor.stehe
07.05.2018
SPD-Antrag Nr. 15: Aufhebung der Straßenbeitragssatzung
Sehr geehrte Frau Bott, ich bitte Sie, folgenden Antrag auf die Tagesordnung der nächsten Sitzung der Stadtverordnetenversammlung zu setzen.
Wir beantragen, die Stadtverordnetenversammlung möge beschließen:
Die Straßenbeitragssatzung der Stadt Griesheim vom 14.12.2012 wird aufgehoben. Eine neue Satzung wird nicht beschlossen.
Begründung:
Erfolgt mündlich Mit freundlichen Grüßen
Fraktionsvorsitzende
SPD-Stadtverordnetenfraktion Griesheim
Fraktionsvorsitzende Ingrid Zimmermann, Goethestr. 30, 64347 Griesheim Tel.: 06155-6914, Email: zimmermann-griesheim@t-onfine.de
=

# Beschlussvorlagen

In [7]:
beschlussvorlagen = data[(data.doc_type == "Beschlussvorlage")]
beschlussvorlagen.head()

# create test dataset for beschlussvorlagen header removal
print("dataset size", len(beschlussvorlagen))
data_set_identifiers = [
    lambda x: "Der Stadtverordnetenversammlung wird folgende Beschlussfassung empfohlen:" in x
]
selected_ids = []
ids = []
for identifier in data_set_identifiers:
    for id, content in zip(beschlussvorlagen.document_id.values, beschlussvorlagen.content.values):
        for line in content:
             if identifier(line):
                ids.append(id)
                break
    print(ids)
    selected_ids += np.random.choice(ids, size=1).tolist()

selected_ids.append(27230) # special case: bürgermeister
print(f"test set: {selected_ids}")

dataset size 56
[3790, 1215, 1216, 496, 3004, 2716, 2705, 1693, 2703, 3993, 432, 1203, 1056, 4643, 5884, 4885, 20076, 17398, 27202, 27230, 26910, 21879, 31737, 29128, 28567, 28541, 28569, 28568, 28570, 28571, 28566, 28695, 36200, 28972, 28931, 28940, 43073]
test set: [2705, 27230]


In [8]:
selector = np.isin(beschlussvorlagen.document_id, selected_ids)
for doc_id, content in zip(beschlussvorlagen.document_id[selector], beschlussvorlagen.content[selector]):
    print(doc_id)
    result = remove_by_regexes(content, [
        r"die Stadtverordnetenversammlung möge beschließen[\s\:\.,]*",
        r"wird folgende Beschlussfassung empfohlen[\s\:\.,]*",
    ])
    
    print("".join(result)[:100], "...")
    print("")
    print("\n".join(content))

    print("=-----------------------------------------==")

2705
Die 2. Änderungssatzung zur Wasserversorgungssatzung (WVS) vom 17.12.2012 wird beschlossen. Sie trit ...

Vorlage an die Stadtverordnetenversammlung der Stadt Griesheim
Beschlussvorlage Steuer- und Gebührenamt öffentlich
Wasserversorgung, 2. Änderungssatzung zur Wasserversorgungssatzung (WVS)
Beratungsfolge Magistrat Wirtschafts- und Finanzausschuss Stadtverordnetenversammlung 17.10.2016 03.11.2016
BV/2016/0172
nicht öffentlich Vorberatung öffentlich Vorberatung öffentlich Entscheidung Der Stadtverordnetenversammlung wird folgende Beschlussfassung empfohlen:
Die 2. Änderungssatzung zur Wasserversorgungssatzung (WVS) vom 17.12.2012 wird beschlossen. Sie tritt am Tage nach ihrer Bekanntmachung in Kraft.
Begründung / Erläuterungen: Aufgrund des entsprechenden Auftrags hat die Firma Schüllermann und Partner AG in Dreieich die Höhe des Wasserbeitrags neu kalkuliert. Da der gegenwärtige Wasserbeitrag bereits seit mehreren Jahren unverändert ist, ist eine Aktualisierung des Beitrags vor 

# Informationsvorlagen

In [16]:
informationsvorlagen = data[(data.doc_type == "Informationsvorlage")]
informationsvorlagen.head()

# create test dataset for beschlussvorlagen header removal
print("dataset size", len(informationsvorlagen))
data_set_identifiers = [
    lambda x: "Die Stadtverordnetenversammlung wird über folgendes Thema informiert:" in x
]
selected_ids = []
ids = []
for identifier in data_set_identifiers:
    for id, content in zip(informationsvorlagen.document_id.values, informationsvorlagen.content.values):
        for line in content:
            if identifier(line):
               ids.append(id)
               break
    print(ids)
    selected_ids += np.random.choice(ids, size=1).tolist()

print(f"test set: {selected_ids}")

dataset size 27
[2806, 3943, 698, 1180, 7053, 11550, 4381, 40421, 46995, 47177, 47158]
test set: [4381]


In [18]:
selector = np.isin(informationsvorlagen.document_id, selected_ids)
for doc_id, content in zip(informationsvorlagen.document_id[selector], informationsvorlagen.content[selector]):
    print(doc_id)
    result = remove_by_regexes(content, [
        r"Die Stadtverordnetenversammlung wird über folgendes Thema informiert:[\s\:\.,]*",
    ])
    
    print("".join(result)[:100], "...")
    print("")
    print("\n".join(content))

    print("=-----------------------------------------==")

4381
Ein Antwortschreiben des Vorstandes der Frankfurter Volksbank zur Resolution liegt vor und ist als A ...

Vorlage an die Stadtverordnetenversammlung der Stadt Griesheim
Informationsvorlage Fachbereich 1 Zentrale Dienste öffentlich
Resolution zum Erhalt der Filiale der Frankfurter Volksbank in der Bessunger Straße
Beratungsfolge Stadtverordnetenversammlung 11.05.2017 öffentlich
IV/2017/0059
Kenntnisnahme Die Stadtverordnetenversammlung wird über folgendes Thema informiert:
Ein Antwortschreiben des Vorstandes der Frankfurter Volksbank zur Resolution liegt vor und ist als Anlage beigefügt.
Begründung / Erläuterungen: In der letzten Stadtverordnetenversammlung wurde eine Resolution zum Erhalt der Filiale der Frankfurter Volksbank in der Bessunger Straße beschlossen. Diese wurde dem Vorstand der Frankfurter Volksbank übermittelt.
Produkt:01.01.02 Städtische Gremien Anlagen: Resolution VOBA Schreiben Griesheim, den 03.05.2017 gez.
Geza Krebs-Wetzl Bürgermeister
Seite 1 von 1
=----------